In [2]:
import sys
from datetime import datetime, timedelta

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.token import TOKEN

In [24]:
with Client(TOKEN) as client:
    for candle in client.get_all_candles(figi="BBG0039320N9", from_=datetime.utcnow() - timedelta(days=2), interval=CandleInterval.CANDLE_INTERVAL_15_MIN):
        print(candle)

HistoricCandle(open=Quotation(units=201, nano=440000000), high=Quotation(units=201, nano=980000000), low=Quotation(units=201, nano=440000000), close=Quotation(units=201, nano=440000000), volume=150, time=datetime.datetime(2022, 2, 14, 7, 30, tzinfo=datetime.timezone.utc), is_complete=True)
HistoricCandle(open=Quotation(units=200, nano=770000000), high=Quotation(units=200, nano=770000000), low=Quotation(units=198, nano=560000000), close=Quotation(units=198, nano=560000000), volume=108, time=datetime.datetime(2022, 2, 14, 9, 0, tzinfo=datetime.timezone.utc), is_complete=True)
HistoricCandle(open=Quotation(units=196, nano=960000000), high=Quotation(units=196, nano=960000000), low=Quotation(units=196, nano=960000000), close=Quotation(units=196, nano=960000000), volume=1, time=datetime.datetime(2022, 2, 14, 9, 30, tzinfo=datetime.timezone.utc), is_complete=True)
HistoricCandle(open=Quotation(units=196, nano=960000000), high=Quotation(units=197, nano=110000000), low=Quotation(units=196, nano

In [5]:
import numpy as np              
import matplotlib.pyplot as plt 
%matplotlib inline 
import pandas as pd            


url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents_symbols.txt"
companies_names = pd.read_csv(url)

In [6]:
companies_names

ABT
0    ABBV
1    ABMD
2     ACN
3    ATVI
4    ADBE
..    ...
499   YUM
500  ZBRA
501   ZBH
502  ZION
503   ZTS

[504 rows x 1 columns]

In [7]:
requests_for_figi = []

for name in companies_names.to_numpy()[:, 0]:
    request_item = {}
    request_item['idType'] = 'TICKER'
    request_item['idValue'] = name
    request_item['exchCode'] = 'US'
    requests_for_figi.append(request_item)

In [8]:
requests_for_figi[:5]

[{'idType': 'TICKER', 'idValue': 'ABBV', 'exchCode': 'US'},
 {'idType': 'TICKER', 'idValue': 'ABMD', 'exchCode': 'US'},
 {'idType': 'TICKER', 'idValue': 'ACN', 'exchCode': 'US'},
 {'idType': 'TICKER', 'idValue': 'ATVI', 'exchCode': 'US'},
 {'idType': 'TICKER', 'idValue': 'ADBE', 'exchCode': 'US'}]

In [9]:
requests_for_figi = np.asarray(requests_for_figi)
requests_for_figi.shape

(504,)

In [10]:
MAX_ALLOWED = 10

In [11]:
batches = [requests_for_figi[i:i + MAX_ALLOWED] for i in range(0, len(requests_for_figi), MAX_ALLOWED)]

In [12]:
len(batches)

51

In [13]:
batches[0]

array([{'idType': 'TICKER', 'idValue': 'ABBV', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'ABMD', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'ACN', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'ATVI', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'ADBE', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'AMD', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'AAP', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'AES', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'AFL', 'exchCode': 'US'},
       {'idType': 'TICKER', 'idValue': 'A', 'exchCode': 'US'}],
      dtype=object)

In [14]:
import requests
import time

In [17]:
openfigi_url = 'https://api.openfigi.com/v1/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

companies_dataset = []
for i, request_batch in enumerate(batches):
    print(f'Batch #{i}')
    
    response = requests.post(url=openfigi_url, headers=openfigi_headers, json=request_batch.tolist())
    while response.status_code == 429:
        print("Request limit T.T")
        time.sleep(60)
        response = requests.post(url=openfigi_url, headers=openfigi_headers, json=request_batch.tolist())
        
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
        
    companies_dataset.append(response.json())

len(companies_dataset)

Batch #0
Batch #1
Batch #2
Batch #3
Batch #4
Batch #5
Request limit T.T
Batch #6
Batch #7
Batch #8
Batch #9
Batch #10
Request limit T.T
Batch #11
Batch #12
Batch #13
Batch #14
Batch #15
Request limit T.T
Request limit T.T
Batch #16
Batch #17
Batch #18
Batch #19
Batch #20
Request limit T.T
Batch #21
Batch #22
Batch #23
Batch #24
Batch #25
Request limit T.T
Batch #26
Batch #27
Batch #28
Batch #29
Batch #30
Request limit T.T
Request limit T.T
Batch #31
Batch #32
Batch #33
Batch #34
Batch #35
Request limit T.T
Batch #36
Batch #37
Batch #38
Batch #39
Batch #40
Request limit T.T
Request limit T.T
Batch #41
Batch #42
Batch #43
Batch #44
Batch #45
Request limit T.T
Batch #46
Request limit T.T
Batch #47
Batch #48
Batch #49
Batch #50


51

In [18]:
companies_dataset[0]

[{'data': [{'figi': 'BBG0025Y4RY4',
    'name': 'ABBVIE INC',
    'ticker': 'ABBV',
    'exchCode': 'US',
    'compositeFIGI': 'BBG0025Y4RY4',
    'uniqueID': None,
    'securityType': 'Common Stock',
    'marketSector': 'Equity',
    'shareClassFIGI': 'BBG0025Y4RZ3',
    'uniqueIDFutOpt': None,
    'securityType2': 'Common Stock',
    'securityDescription': 'ABBV'}]},
 {'data': [{'figi': 'BBG000C101X4',
    'name': 'ABIOMED INC',
    'ticker': 'ABMD',
    'exchCode': 'US',
    'compositeFIGI': 'BBG000C101X4',
    'uniqueID': None,
    'securityType': 'Common Stock',
    'marketSector': 'Equity',
    'shareClassFIGI': 'BBG001S64SQ9',
    'uniqueIDFutOpt': None,
    'securityType2': 'Common Stock',
    'securityDescription': 'ABMD'}]},
 {'data': [{'figi': 'BBG000D9D830',
    'name': 'ACCENTURE PLC-CL A',
    'ticker': 'ACN',
    'exchCode': 'US',
    'compositeFIGI': 'BBG000D9D830',
    'uniqueID': None,
    'securityType': 'Common Stock',
    'marketSector': 'Equity',
    'shareClassFI

In [19]:
companies = [item for sublist in companies_dataset for item in sublist]

companies_flat = []
for item in companies:
    if 'data' in item:
        companies_flat.append(item['data'][0])

In [20]:
companies_as_df = pd.DataFrame(companies_flat)

In [21]:
companies_as_df

figi                          name ticker exchCode compositeFIGI  \
0    BBG0025Y4RY4                    ABBVIE INC   ABBV       US  BBG0025Y4RY4   
1    BBG000C101X4                   ABIOMED INC   ABMD       US  BBG000C101X4   
2    BBG000D9D830            ACCENTURE PLC-CL A    ACN       US  BBG000D9D830   
3    BBG000CVWGS6       ACTIVISION BLIZZARD INC   ATVI       US  BBG000CVWGS6   
4    BBG000BB5006                     ADOBE INC   ADBE       US  BBG000BB5006   
..            ...                           ...    ...      ...           ...   
496  BBG000BH3GZ2               YUM! BRANDS INC    YUM       US  BBG000BH3GZ2   
497  BBG000CC7LQ7  ZEBRA TECHNOLOGIES CORP-CL A   ZBRA       US  BBG000CC7LQ7   
498  BBG000BKPL53    ZIMMER BIOMET HOLDINGS INC    ZBH       US  BBG000BKPL53   
499  BBG000BX9WL1              ZIONS BANCORP NA   ZION       US  BBG000BX9WL1   
500  BBG0039320N9                    ZOETIS INC    ZTS       US  BBG0039320N9   

    uniqueID  securityType marketSector shareClassFIGI uniqueIDFutOpt  \
0       None  Common Stock       Equity   BBG0025Y4RZ3           None   
1       None  Common Stock       Equity   BBG001S64SQ9           None   
2       None  Common Stock       Equity   BBG001SCXK90           None   
3       None  Common Stock       Equity   BBG001S6C009           None   
4       None  Common Stock       Equity   BBG001S5NCQ5           None   
..       ...           ...          ...            ...            ...   
496     None  Common Stock       Equity   BBG001S7JQ30           None   
497     None  Common Stock       Equity   BBG001S6SX73           None   
498     None  Common Stock       Equity   BBG001S7DQJ9           None   
499     None  Common Stock       Equity   BBG001S5XN80           None   
500     None  Common Stock       Equity   BBG0039320P7           None   

    securityType2 securityDescription  
0    Common Stock                ABBV  
1    Common Stock                ABMD  
2    Common Stock                 ACN  
3    Common Stock                ATVI  
4    Common Stock                ADBE  
..            ...                 ...  
496  Common Stock                 YUM  
497  Common Stock                ZBRA  
498  Common Stock                 ZBH  
499  Common Stock                ZION  
500  Common Stock                 ZTS  

[501 rows x 12 columns]

In [22]:
companies_as_df.to_csv('companies_dataset.csv', index=False)

In [1]:
 companies_as_df = pd.read_csv('companies_dataset.csv')

NameError: name 'pd' is not defined